# Project 2 - Source coding, data compression and channel coding

The goal of this second project is to apply some of the principles seen in the lectures about source coding, data compression and channel coding. We ask you to write a brief report (pdf format) collecting your answers to the different questions. All codes must be written in Python inside this Jupyter. Note that you can not change the content of locked cells or import any extra Python library than the ones already imported (numpy).

## Implementation

In this project, you will need to use implement source coding algorithms to answer several questions. Therefore, in this first part, you are asked to write several functions that implement two of the  algorithms seen in the theoretical lectures and one new algorithm described in the project statement. Remember that you need to fill in this Jupyter Notebook to answer these questions. Pay particular attention to the required input and output format of each function.

In [1]:
# [Locked Cell] You can not import any extra Python library in this Notebook.
import numpy as np

### Question 1
Implement a function that returns a binary Huffman code for a given probability distribution. Give the main steps of your implementation. Verify your code on Exercise 7 of the second exercise session (TP2), and report the output of your code for this example. Explain how to extend your function to generate a Huffman code of any (output) alphabet size. 


In [2]:
class Node:
    """
    Node class for Huffman tree
    """
    def __init__(self, probability, symbol, leftChild=None, rightChild=None):
        self.probability = probability
        self.leftChild = leftChild
        self.rightChild = rightChild
        self.symbol = symbol

def get_codeword(node, codeword):
    """
    Recursively get the codeword for each node in the Huffman tree
    
    Arguments:
    ----------
    node: root node of the Huffman tree
    codeword: codeword already built (accumulator)
    
    Return:
    -------
    - codewords: dictionary with the name and the corresponding codeword 
      - keys: symbol as character or string
      - values: associated codeword as a character or a string    
      Example: {"A": "10", "B":"0","C":"111","D":"110"}
    
    """
    if node.leftChild is None and node.rightChild is None:
        return {node.symbol: codeword}
    else:
        codeword_left = codeword + "0"
        codeword_right = codeword + "1"
        codewords = get_codeword(node.leftChild, codeword_left)
        codewords.update(get_codeword(node.rightChild, codeword_right))
        return codewords

def Huffman_code(probability_dict):
    """
    Create the Huffman code for given probabilities  
    
    Arguments:
    ----------
    probability_dict:
      - keys: symbol as character or string
      - values: probability of the symbol as a float or double
      Example: {"A": 0.25, "B":0.5,"C":0.125,"D":0.125}
    
    Return:
    -------
    - codewords: dictionary with the name and the corresponding codeword 
      - keys: symbol as character or string
      - values: associated codeword as a character or a string    
      Example: {"A": "10", "B":"0","C":"111","D":"110"}
    
    """ 
    # Create the nodes of the tree from the probabilities
    nodes = []
    for symbol, probability in probability_dict.items():
        nodes.append(Node(probability, symbol))

    # Sort the list of Node objects by probability
    nodes.sort(key=lambda x: x.probability)

    # Create the Huffman tree
    while len(nodes) > 1:
        left = nodes.pop(0)
        right = nodes.pop(0)
        parent = Node(left.probability + right.probability, None, left, right)
        nodes.append(parent)
        nodes.sort(key=lambda x: x.probability)

    # Get the codewords on the root of the tree
    codewords = get_codeword(nodes[0], "")
    return codewords

### Question 2

Given a sequence of symbols, implement a function that returns a dictionary and the encoded sequence using the on-line Lempel-Ziv algorithm (see State of the art in data compression, slide 50/53). Reproduce and report the example given in the course. 

In [ ]:

def LZ_online(sequence):
    """
    The on-line Lempel-Ziv algorithm given a sequence of symbols  
    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format
    
    Return:
    -------
    - dictionary : the computed dictionnary in the form:
        - keys: symbol as character or string
        - values: associated codeword as a tuple composed of the entry index (integer) and a binarized adress with one appended symbol (character or string)
        Example: {'': (0, ''), '0': (1, '0'), '1': (2, '01'), '00': (3, '010'), '10': (4, '100')}
    - encoded_sequence : the encoded sequence in the string format
    """

### Question 4

Implement a function that returns the encoded sequence using the LZ77 algorithm as described by the algorithm below given an input string and a sliding window size l. Reproduce the example given in Figure 2 with window_size=7.

In [ ]:
def LZ77(sequence, window_size=7):
    """
    The Lempel-Ziv 77 algorithm given a sequence of symbols and the sliding window size
    
    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format
    - window_size : sliding window size as an integer
    
    Return:
    -------
    - encoded_sequence : the encoded sequence in the string format
    """

In [ ]:
# [Locked Cell] Evaluation of your functions by the examiner. 
# You don't have access to the evaluation, this will be done by the examiner.
# Therefore, this cell will return nothing for the students.
import os
if os.path.isfile("private_evaluation.py"):
    from private_evaluation import unit_tests
    unit_tests(Huffman_code, LZ_online, LZ77)

## Source coding and reversible (lossless) data compression


In [3]:
# Read Morse file
morseFile = open("data/morse.txt", "r")
morseText = morseFile.read()
morseFile.close()
print("Length of Morse text = " + str(len(morseText)) + " symbols")

# Read text file
textFile = open("data/text.txt", "r")
text = textFile.read()
textFile.close()
print("Length of text = " + str(len(text)) + " symbols")

Length of Morse text = 1199290 symbols
Length of text = 412310 symbols


### Question 5

In [5]:
# Compute marginal probability distribution
probas = {}
for symbol in morseText:
    symb = probas.get(symbol)
    if symb == None:
        probas[symbol] = 1
    else:
        probas[symbol] = symb + 1
for key in probas.keys():
    probas[key]/=len(morseText)

print("Probabilities: ", end="")
print(probas)

# Binary Huffman code
binHuffmanCode = Huffman_code(probas)
print("Huffman code: ", end="")
print(binHuffmanCode)

# Encode text with Huffman code
encodedMorseText = ""
for symbol in morseText:
    encodedMorseText+=binHuffmanCode[symbol]
#print(encodedMorseText)

# Total length of encoded Morse text
print("Length of encoded morse text = " + str(len(encodedMorseText)) + " bits")
print("Length of morse text = " + str(len(morseText)*4) + " bits") # *4 because 4 different symbols

# Compression rate
compressionRate = (len(morseText)*4)/len(encodedMorseText)
print("Compression rate = " + str(compressionRate))

Probabilities: {'.': 0.43378082031868853, '_': 0.2145202578192097, '-': 0.2870623452209224, '/': 0.06463657664117936}
Huffman code: {'.': '0', '/': '100', '_': '101', '-': '11'}
Length of encoded morse text = 2213141 bits
Length of morse text = 4797160 bits
Compression rate = 2.1675799237373488


### Question 6

In [11]:
# Expected average length of Huffman code
avgLengthHuffman = 0.0
for key in probas.keys():
    avgLengthHuffman+=probas[key]*len(binHuffmanCode[key])
print("Expected average length of Huffman code = " + str(avgLengthHuffman))

# Emperical = ratio length encoded and length initial
emperical = len(encodedMorseText) / len(morseText)
print("Emperical average length of Huffman code = " + str(emperical))

# Entropy
entropy = 0.0
for key in probas.keys():
    entropy+=probas[key]*np.log2(probas[key])

print("H(S)/log2(q) = " + str(-entropy)) # log2(2) = 1
print("(H(S)/log2(q)) + 1 = " + str(-entropy+1))

Expected average length of Huffman code = 1.8453760141417006
Emperical average length of Huffman code = 1.8453760141417006
H(S)/log2(q) = 1.7713848706432271
(H(S)/log2(q)) + 1 = 2.771384870643227


### Question 7

In [47]:
# Compute data
codes = {}
empericalAverages = {}
for inputLength in range(1, len(morseText)):
    if len(morseText) % inputLength == 0:
        indexStart = 0
        indexEnd = inputLength - 1
        nbBlocks = 0.0
        probas = {}
        while indexEnd < len(morseText):
            subText = morseText[indexStart : indexEnd+1]
            symb = probas.get(subText)
            if symb == None:
                probas[subText] = 1
            else:
                probas[subText] = symb + 1
            nbBlocks+=1.0
            indexStart += inputLength
            indexEnd += inputLength
        for key in probas.keys():
            probas[key]/=nbBlocks
        codes[inputLength] = Huffman_code(probas)
        
        encodedMorseText = ""
        indexStart = 0
        indexEnd = inputLength - 1
        while indexEnd < len(morseText):
            encodedMorseText+=codes[inputLength][morseText[indexStart : indexEnd+1]]
            indexStart += inputLength
            indexEnd += inputLength
        empericalAverages[inputLength] = len(encodedMorseText) / len(morseText)
print(empericalAverages)

{1: 1.8453760141417006, 2: 1.709244636409876, 5: 1.5083299285410534, 10: 1.245440218796121, 119929: 2.8350107146728482e-05, 239858: 1.0005920169433581e-05, 599645: 1.6676533615722636e-06}


## Channel coding

In [ ]:
# Write here your codes for questions 16 to 21 (you may delete this comment)
# From here, you may import either opencv (cv2) or the Python Imaging Library (PIL), but no other extra libraries.